In [1]:
!java -version



openjdk version "17.0.16" 2025-07-15
OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test").getOrCreate()
spark


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/30 15:41:48 WARN Utils: Your hostname, nadhifa, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/11/30 15:41:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/30 15:41:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Example: Linear Regression with Spark MLlib
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

# Initialize Spark Session
spark = SparkSession.builder.appName('MLlib Example').getOrCreate()

# Load sample data
data = [(1, 5.0, 20.0), (2, 10.0, 25.0), (3, 15.0, 30.0), (4, 20.0, 35.0)]
columns = ['ID', 'Feature', 'Target']
df = spark.createDataFrame(data, columns)

# Prepare data for modeling
assembler = VectorAssembler(inputCols=['Feature'], outputCol='Features')
df_transformed = assembler.transform(df)

# Train a linear regression model
lr = LinearRegression(featuresCol='Features', labelCol='Target')
model = lr.fit(df_transformed)

# Print model coefficients
print(f'Coefficients: {model.coefficients}')
print(f'Intercept: {model.intercept}')


25/11/30 15:50:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/11/30 15:50:32 WARN Instrumentation: [6ca14445] regParam is zero, which might cause numerical instability and overfitting.
25/11/30 15:50:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/11/30 15:50:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Coefficients: [0.9999999999999992]
Intercept: 15.000000000000009


In [6]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import Vectors

# Data: gunakan Vectors.dense untuk kolom fitur, label harus numeric (float atau int)
data = [
    (1, Vectors.dense([2.0, 3.0]), 0.0),
    (2, Vectors.dense([1.0, 5.0]), 1.0),
    (3, Vectors.dense([2.5, 4.5]), 1.0),
    (4, Vectors.dense([3.0, 6.0]), 0.0)
]
columns = ['ID', 'Features', 'Label']

df = spark.createDataFrame(data, columns)
df.printSchema()
df.show(truncate=False)

# Train logistic regression model
lr = LogisticRegression(featuresCol='Features', labelCol='Label', maxIter=20)
model = lr.fit(df)

# Display coefficients and intercept
print('Coefficients:', model.coefficients)
print('Intercept:', model.intercept)



root
 |-- ID: long (nullable = true)
 |-- Features: vector (nullable = true)
 |-- Label: double (nullable = true)

+---+---------+-----+
|ID |Features |Label|
+---+---------+-----+
|1  |[2.0,3.0]|0.0  |
|2  |[1.0,5.0]|1.0  |
|3  |[2.5,4.5]|1.0  |
|4  |[3.0,6.0]|0.0  |
+---+---------+-----+

Coefficients: [-9.352581579369746,3.1168035677591837]
Intercept: 8.66316853737857


In [8]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors

data = [
    (1, Vectors.dense([1.0, 1.0])),
    (2, Vectors.dense([5.0, 5.0])),
    (3, Vectors.dense([10.0,10.0])),
    (4, Vectors.dense([15.0,15.0]))
]
df = spark.createDataFrame(data, ["ID", "Features"])
df.printSchema()
df.show(truncate=False)

kmeans = KMeans(featuresCol='Features', k=2)
model = kmeans.fit(df)
print("Cluster Centers:", model.clusterCenters())



root
 |-- ID: long (nullable = true)
 |-- Features: vector (nullable = true)

+---+-----------+
|ID |Features   |
+---+-----------+
|1  |[1.0,1.0]  |
|2  |[5.0,5.0]  |
|3  |[10.0,10.0]|
|4  |[15.0,15.0]|
+---+-----------+

Cluster Centers: [array([12.5, 12.5]), array([3., 3.])]


In [9]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Titanic-MLlib-Homework").getOrCreate()

# Imports umum
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


25/11/30 16:19:42 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [11]:
# Homework 1 oad a real-world dataset into Spark and prepare it for machine learning tasks.
# 1. Load Dataset
path = "/home/nadhifa/Downloads/archive/Titanic-Dataset.csv"
df = spark.read.csv(path, header=True, inferSchema=True)

# 2. Pilih kolom relevan dan cek
df = df.select("Survived","Pclass","Sex","Age","SibSp","Parch","Fare","Embarked")
df.printSchema()
df.show(5)

# 3. Tangani missing: drop atau imputasi sederhana
# Untuk praktikum sederhana kita hapus baris yang punya Age atau Fare null
df = df.dropna(subset=['Age','Fare'])

# Untuk Embarked yang sedikit missing, imputasi modus:
most_freq = df.groupBy("Embarked").count().orderBy(col("count").desc()).first()['Embarked']
df = df.fillna({'Embarked': most_freq})

# 4. Encode kategori: Sex dan Embarked
sex_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex', handleInvalid='keep')
emb_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbIndex', handleInvalid='keep')

# 5. (Optional) OneHot jika mau: contoh pakai OneHotEncoder untuk Embarked
emb_ohe = OneHotEncoder(inputCols=['EmbIndex'], outputCols=['EmbOHE'], handleInvalid='keep')

# 6. Buat features vector
assembler = VectorAssembler(
    inputCols=['Pclass','SexIndex','Age','SibSp','Parch','Fare','EmbOHE'],
    outputCol='features', handleInvalid='keep'
)

# 7. Pipeline preprocessing
pipeline_prep = Pipeline(stages=[sex_indexer, emb_indexer, emb_ohe, assembler])
prep_model = pipeline_prep.fit(df)
df_prepared = prep_model.transform(df)

# 8. Final dataset untuk modelling: features + label (Pastikan label numeric double)
df_ml = df_prepared.select(col('features'), col('Survived').cast('double').alias('label'))
df_ml.show(5, truncate=False)
print("Total rows after prepare:", df_ml.count())


root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 5 rows
+----------------------------------------------+-----+
|features                                      |label|
+--------------------------

In [12]:
# Homework 2 Build a classification model using Spark MLlib and evaluate its performance.

# train, test = df_ml.randomSplit([0.8, 0.2], seed=42)

# Definisikan model (Logistic Regression)
lr = LogisticRegression(featuresCol='features', labelCol='label', maxIter=50)

# 3. Fit model
lr_model = lr.fit(train)

# 4. Prediksi di test
pred = lr_model.transform(test)

# 5. Evaluator: AUC (binary) dan accuracy (multiclass evaluator)
bce = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='rawPrediction', metricName='areaUnderROC')
auc = bce.evaluate(pred)
mce = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = mce.evaluate(pred)

print(f"AUC (ROC): {auc:.4f}")
print(f"Accuracy: {accuracy:.4f}")

# 6. Confusion matrix kecil (kumpulkan ke pandas untuk tampilan)
cm = pred.groupBy('label','prediction').count().toPandas()
print("\nConfusion counts (label, prediction, count):")
print(cm)


AUC (ROC): 0.9048
Accuracy: 0.7982

Confusion counts (label, prediction, count):
   label  prediction  count
0    1.0         1.0     38
1    0.0         1.0     10
2    1.0         0.0     13
3    0.0         0.0     53


In [14]:
# Homework 3 Explore hyperparameter tuning using cross-validation.

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# 1. Siapkan estimator & evaluator
lr = LogisticRegression(featuresCol='features', labelCol='label', maxIter=50)
evaluator = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='rawPrediction', metricName='areaUnderROC')

# 2. Buat grid parameter
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.1, 1.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .build())

# 3. CrossValidator (gunakan numFolds=3 atau 4 sesuai waktu)
cv = CrossValidator(estimator=lr,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3,
                    parallelism=2)  # parallelism = berapa model yang dilatih paralel

# 4. Fit CV (hati-hati: ini butuh waktu)
cvModel = cv.fit(train)

# 5. Prediksi & evaluasi model terbaik
bestModel = cvModel.bestModel
print("Best RegParam:", bestModel._java_obj.getRegParam())
print("Best ElasticNetParam:", bestModel._java_obj.getElasticNetParam())

pred_cv = cvModel.transform(test)
auc_cv = evaluator.evaluate(pred_cv)
acc_cv = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy').evaluate(pred_cv)
print(f"AUC after CV: {auc_cv:.4f}, Accuracy after CV: {acc_cv:.4f}")


Best RegParam: 0.01
Best ElasticNetParam: 1.0
AUC after CV: 0.9038, Accuracy after CV: 0.7895
